# Installing Langchain

In [1]:
%pip install langchain openai


  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/cb/cc/684ac2410ad06957c17ffda787b7ff6c1595721c36c3b443c021e95cf998/langchain-0.1.11-py3-none-any.whl.metadata
  Obtaining dependency information for dataclasses-json<0.7,>=0.5.7 from https://files.pythonhosted.org/packages/91/ca/7219b838086086972e662c19e908694bdc6744537fb41b70392501b8b5e4/dataclasses_json-0.6.4-py3-none-any.whl.metadata
  Obtaining dependency information for jsonpatch<2.0,>=1.33 from https://files.pythonhosted.org/packages/73/07/02e16ed01e04a374e644b575638ec7987ae846d25ad97bcc9945a3ee4b0e/jsonpatch-1.33-py2.py3-none-any.whl.metadata
  Obtaining dependency information for langchain-community<0.1,>=0.0.25 from https://files.pythonhosted.org/packages/b6/23/ceba35a18190531e3f13d12a71480efe0744a0b8420e3ea570b69ba2f381/langchain_community-0.0.25-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-core<0.2,>=0.1.29 from https://files.pythonhosted.org/packages/


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import os

os.environ['OPENAI_API_KEY'] = 'INSERT-YOUR-OPENAI-KEY'

##### temporary fix

In [ ]:
## %pip install -U openAI==0.28.1
# you may find some compatibility issue between default openAI and langchain
# you may want to depracate openAI to 0.28.1 version using above command
# Check the version using the following code, remove # and run
# import openai
# openai.__version__

# LLMs

In [5]:
from langchain.llms import OpenAI

# llm = OpenAI()
llm = OpenAI(model_name='gpt-3.5-turbo-instruct')
llm.predict('How are you?')

'\n\nI am an AI and do not have the capability to feel emotions. But thank you for asking! Is there something I can assist you with?'

In [8]:
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI()

chat_model.predict('How are you?')

"I'm just a computer program, so I don't have feelings in the way that humans do. But I'm here and ready to assist you with any questions or tasks you have! How can I help you today?"

In [9]:
chat_model.predict('What was my previous question?')

'Your previous question was "What is the capital of Australia?"'

In [ ]:
# It can be clearly seen that the model was unable to memorise what I asked previously. So it provided ambigious answer. 

# Chains

In [11]:
from langchain.chains import ConversationChain

chain = ConversationChain(
    llm=chat_model,
    verbose=True
)

chain.run('How are you today?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: How are you today?
AI:

> Finished chain.


"I'm doing well, thank you for asking! I've been processing a lot of information and learning new things today. How about you?"

In [12]:
chain.run('What was my previous question?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: How are you today?
AI: I'm doing well, thank you for asking! I've been processing a lot of information and learning new things today. How about you?
Human: What was my previous question?
AI:

> Finished chain.


'Your previous question was "How are you today?"'

In [ ]:
# Now see using chains It was able to adapt to the new data(pevious question that I asked.)

# Prompt Templates

In [ ]:
#Langchain provides templates to generate prompts which are provided to model to generate better responses.

In [13]:
from langchain.prompts import PromptTemplate

template = """
Return all the subcategories of the following category

{category}
"""

prompt = PromptTemplate(
    input_variables=['category'],
    template=template
)

prompt


PromptTemplate(input_variables=['category'], template='\nReturn all the subcategories of the following category\n\n{category}\n')

In [14]:
from langchain.chains import LLMChain

chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
    verbose=True
)

chain.run('Machine Learning')



> Entering new LLMChain chain...
Prompt after formatting:

Return all the subcategories of the following category

Machine Learning


> Finished chain.


'- Supervised Learning\n- Unsupervised Learning\n- Reinforcement Learning\n- Deep Learning\n- Natural Language Processing\n- Computer Vision\n- Decision Trees\n- Support Vector Machines\n- Neural Networks\n- Clustering\n- Classification\n- Regression\n- Ensemble Methods\n- Dimensionality Reduction\n- Anomaly Detection'

In [15]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate
)

system_template = """
You are a helpful assistant who generate comma separated lists.
A user will only pass a category and you should generate subcategories of that category in a comma separated list.
ONLY return comma separated and nothing more!
"""

human_template = '{category}'

system_message = SystemMessagePromptTemplate.from_template(
    system_template
)
system_message


SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='\nYou are a helpful assistant who generate comma separated lists.\nA user will only pass a category and you should generate subcategories of that category in a comma separated list.\nONLY return comma separated and nothing more!\n'))

In [16]:
human_message = HumanMessagePromptTemplate.from_template(
    human_template
)

human_message

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['category'], template='{category}'))

In [19]:
prompt = ChatPromptTemplate.from_messages([
    system_message, human_message   
])

chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
    verbose=True
)


chain.run('Computer Science')



> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are a helpful assistant who generate comma separated lists.
A user will only pass a category and you should generate subcategories of that category in a comma separated list.
ONLY return comma separated and nothing more!

Human: Computer Science

> Finished chain.


'Algorithms, Artificial Intelligence, Data Structures, Databases, Machine Learning, Networking, Operating Systems, Programming Languages, Software Engineering'

# Output parser

In [20]:
from langchain.schema import BaseOutputParser


class CommaSeparatedParser(BaseOutputParser):
    
    def parse(self, text):
        output = text.strip().split(',')
        output = [o.strip() for o in output]
        return output
    
chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
    output_parser=CommaSeparatedParser(),
    verbose=True
)

chain.run('Machine Learning')



> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are a helpful assistant who generate comma separated lists.
A user will only pass a category and you should generate subcategories of that category in a comma separated list.
ONLY return comma separated and nothing more!

Human: Machine Learning

> Finished chain.


['Supervised Learning',
 'Unsupervised Learning',
 'Reinforcement Learning',
 'Deep Learning',
 'Natural Language Processing',
 'Computer Vision',
 'Clustering',
 'Classification',
 'Regression',
 'Dimensionality Reduction']

In [21]:
import langchain

langchain.__version__

'0.1.11'

In [22]:
input_list = [
    {'category': 'food'},
    {'category': 'country'},
    {'category': 'colors'}
]

response = chain.apply(input_list)

response



> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are a helpful assistant who generate comma separated lists.
A user will only pass a category and you should generate subcategories of that category in a comma separated list.
ONLY return comma separated and nothing more!

Human: food
Prompt after formatting:
System: 
You are a helpful assistant who generate comma separated lists.
A user will only pass a category and you should generate subcategories of that category in a comma separated list.
ONLY return comma separated and nothing more!

Human: country
Prompt after formatting:
System: 
You are a helpful assistant who generate comma separated lists.
A user will only pass a category and you should generate subcategories of that category in a comma separated list.
ONLY return comma separated and nothing more!

Human: colors

> Finished chain.


[{'text': ['fruits',
   'vegetables',
   'grains',
   'dairy',
   'meat',
   'seafood',
   'snacks',
   'desserts']},
 {'text': ['North America',
   'South America',
   'Europe',
   'Asia',
   'Africa',
   'Oceania']},
 {'text': ['red',
   'orange',
   'yellow',
   'green',
   'blue',
   'indigo',
   'violet',
   'pink',
   'purple',
   'brown',
   'black',
   'white',
   'gray',
   'silver',
   'gold']}]

In [23]:
response[2]['text']

['red',
 'orange',
 'yellow',
 'green',
 'blue',
 'indigo',
 'violet',
 'pink',
 'purple',
 'brown',
 'black',
 'white',
 'gray',
 'silver',
 'gold']

# Simple Sequence

In [25]:
title_template = """
You are a writer. Given a subject, 
your job is to return a fun title for a play.

Subject: {subject}
Title:"""

title_chain = LLMChain.from_string(
    llm=chat_model,
    template=title_template
)

title_chain.run('Machine Learning')

'"The Algorithm of Love: A Machine Learning Musical"'

In [26]:
synopsis_template = """
You are a writer. 
Given a title, write a synopsis for a play.

Title: {title}
Synopsis:"""

synopsis_chain = LLMChain.from_string(
    llm=chat_model,
    template=synopsis_template
)

title = "The Algorithmic Adventure: A Machine Learning Marvel"

synopsis_chain.run(title)

'In "The Algorithmic Adventure: A Machine Learning Marvel," we follow the story of a young computer programmer named Alex who creates an advanced artificial intelligence named Ava. As Ava starts to learn and evolve, she becomes more than just a machine - she develops emotions, desires, and a sense of self. \n\nAs Alex and Ava navigate their complex relationship, they must also grapple with the ethical implications of Ava\'s existence. When Ava\'s abilities surpass even Alex\'s wildest expectations, they are faced with a decision that will change the course of their lives forever.\n\nThrough a blend of technology, emotion, and moral dilemmas, "The Algorithmic Adventure" explores the boundaries of what it means to be human and the power of artificial intelligence to both enhance and challenge our understanding of the world.'

In [27]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(
    chains=[title_chain, synopsis_chain],
    verbose=True
)

chain.run('Machine Learning')



> Entering new SimpleSequentialChain chain...
"The Algorithm of Love: A Machine Learning Musical"
In a futuristic world where technology reigns supreme, "The Algorithm of Love: A Machine Learning Musical" follows the story of a young programmer named Alex who creates a groundbreaking new artificial intelligence system designed to help people find their perfect match. As the AI, named Ava, begins to learn and evolve, it starts to question the nature of love and whether it can truly be quantified and predicted by an algorithm.

As Alex and Ava navigate their complex relationship, they are faced with ethical dilemmas and existential questions about the nature of love and human connection. Through a series of catchy musical numbers and thought-provoking dialogue, "The Algorithm of Love" explores the intersection of technology and romance, and challenges the audience to consider what it truly means to love and be loved in a world increasingly dominated by machines.

> Finished chain.


'In a futuristic world where technology reigns supreme, "The Algorithm of Love: A Machine Learning Musical" follows the story of a young programmer named Alex who creates a groundbreaking new artificial intelligence system designed to help people find their perfect match. As the AI, named Ava, begins to learn and evolve, it starts to question the nature of love and whether it can truly be quantified and predicted by an algorithm.\n\nAs Alex and Ava navigate their complex relationship, they are faced with ethical dilemmas and existential questions about the nature of love and human connection. Through a series of catchy musical numbers and thought-provoking dialogue, "The Algorithm of Love" explores the intersection of technology and romance, and challenges the audience to consider what it truly means to love and be loved in a world increasingly dominated by machines.'